---

# Assignment 4

Welcome to the last assignment! Here you will evaluate the recommenders you developed in the previous assignemtns. We will use part of the MovieLens 20M dataset.

You will write and execute your code in Python using this Jupyter Notebook.

**PREREQUISITE:** Download the Movie Dataset from: <https://www.kaggle.com/rounakbanik/the-movies-dataset/data>. Extract the contents of the zip file into `data_directory`.
So you should have, for example, a file `data_directory/movies_metadata.csv`.

Also download the MovieLens 20M dataset from <https://grouplens.org/datasets/movielens/20m/>. Extract the `ratings.csv` file and overwrite the one in `data_directory` in the code. You should then have a file `data_directory/ratings.csv` that has about 20M ratings, instead of about 26M contained in the full MovieLens dataset from kaggle.

**TASK:** Your job is: (1) Copy your code from previous assignments into the correct place in the `xxx-TO_EDIT.py` files; rename them to `xxx.py`, once done.
(2) Fill in the missing code in this notebook.
In both cases, the place to enter your code is clearly marked with comments.

**SUBMISSION:** You will submit only this Notebook via TUWEL. Do not submit the `xxx.py` files --- we will use our own.

**GRADING:** We will test whether you code produces the expected output that is provided. We will also run additional tests, not shown here.

## Preparation
Importing necessary modules.

In [1]:
%load_ext autoreload
%autoreload 2

import csv
import pandas as pd
import numpy as np

Make sure to enter the correct location of your data.

In [2]:
data_directory = '../assignment1/data/'

## Create the movies DataFrame

In [3]:
links = pd.read_csv(data_directory + 'links.csv')
movies_plain = pd.read_csv(data_directory + 'movies.csv')
metadata = pd.read_csv(data_directory + 'movies_metadata.csv', low_memory=False)
metadata.drop(metadata.columns[[0,1,2,4,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23]], axis=1, inplace=True)
keywords = pd.read_csv(data_directory + 'keywords.csv', low_memory=False)
credits = pd.read_csv(data_directory + 'credits.csv', low_memory=False)

keywords['id'] = keywords['id'].astype('int')
links=links[links['tmdbId'].isnull()==False]
links['tmdbId'] = links['tmdbId'].astype('int')
metadata = metadata.drop([19730, 29503, 35587])
metadata['id'] = metadata['id'].astype('int')
credits['id'] = credits['id'].astype('int')

movies = metadata.merge(links, how='inner', left_on='id', right_on='tmdbId')
movies = movies.merge(movies_plain, how='inner', left_on='movieId', right_on='movieId')
movies = movies.merge(keywords, how='inner', left_on='id', right_on='id')
movies = movies.merge(credits, how='inner', left_on='id', right_on='id')
movies = movies.drop(columns=['tmdbId','genres_y'])
movies.rename(columns={'genres_x': 'genres'}, inplace=True)

movies=movies[movies['overview'].isnull()==False]

movies = movies[movies['movieId'] <= 1000]

from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    movies[feature] = movies[feature].apply(literal_eval)
    

# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

# Define new director, cast, genres and keywords features that are in a suitable form.
movies['director'] = movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    movies[feature] = movies[feature].apply(get_list)

    
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # Check if string exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    movies[feature] = movies[feature].apply(clean_data)

    
# Drop duplicate movies   
import collections
movie_ids = movies['movieId'].tolist()
movie_ids_dup = [x for  x, y in collections.Counter(movie_ids).items() if y > 1]

for movie_id in movie_ids_dup:
    to_drop = movies.index[movies.movieId == movie_id].tolist()[1:]
    movies.drop(to_drop, inplace=True)

movies.drop(columns='crew', inplace=True)


movies.rename(columns={'overview':'plot'}, inplace=True)

def create_metadata(x):
        return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])  

# Create a new metadata feature
movies['metadata'] = movies.apply(create_metadata, axis=1)

movies.head()

,genres,id,plot,movieId,imdbId,title,keywords,cast,director,metadata
0,"[animation, comedy, family]",862,"Led by Woody, Andy's toys live happily in his ...",1,114709,Toy Story (1995),"[jealousy, toy, boy]","[tomhanks, timallen, donrickles]",johnlasseter,jealousy toy boy tomhanks timallen donrickles ...
1,"[adventure, fantasy, family]",8844,When siblings Judy and Peter discover an encha...,2,113497,Jumanji (1995),"[boardgame, disappearance, basedonchildren'sbook]","[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,boardgame disappearance basedonchildren'sbook ...
2,"[romance, comedy]",15602,A family wedding reignites the ancient feud be...,3,113228,Grumpier Old Men (1995),"[fishing, bestfriend, duringcreditsstinger]","[waltermatthau, jacklemmon, ann-margret]",howarddeutch,fishing bestfriend duringcreditsstinger walter...
3,"[comedy, drama, romance]",31357,"Cheated on, mistreated and stepped on, the wom...",4,114885,Waiting to Exhale (1995),"[basedonnovel, interracialrelationship, single...","[whitneyhouston, angelabassett, lorettadevine]",forestwhitaker,basedonnovel interracialrelationship singlemot...
4,[comedy],11862,Just when George Banks has recovered from his ...,5,113041,Father of the Bride Part II (1995),"[baby, midlifecrisis, confidence]","[stevemartin, dianekeaton, martinshort]",charlesshyer,baby midlifecrisis confidence stevemartin dian...


## Create the ratings DataFrame

In [34]:
ratings = pd.read_csv(data_directory + 'ratings.csv')
ratings = ratings.drop(columns=['timestamp'])
ratings = ratings[(ratings['userId'] < 1000) & (ratings['movieId'] < 100) ]

ratings = ratings[ratings['movieId'].isin(movies['movieId'])]

## keep users with more than 2 ratings
ratings_count = ratings.groupby(['userId', 'movieId']).size().groupby('userId').size()
ratings_ok = ratings_count[ratings_count >= 2].reset_index()[['userId']]
ratings = ratings.merge(ratings_ok, 
               how = 'right',
               left_on = 'userId',
               right_on = 'userId')



userIds = ratings.userId.unique()
userIds.sort()
userId_to_userIDX = dict(zip(userIds, range(0, userIds.size)))

ratings = pd.concat([ratings['userId'].map(userId_to_userIDX), ratings['movieId'], ratings['rating']], axis=1)
ratings.columns = ['user', 'item', 'rating']

ratings.head()

,user,item,rating
0,0,2,3.5
1,0,29,3.5
2,0,32,3.5
3,0,47,3.5
4,0,50,3.5


## Split ratings into train and test subsets

In [5]:
from sklearn.model_selection import train_test_split


ratings_train, ratings_test = train_test_split(ratings,
                                               stratify=ratings['user'],
                                               test_size=0.20,
                                               random_state=42)



### keep only users which have at least one positive (>3) ratings in train
positive_ratings = ratings_train[ratings_train['rating']>3]
positive_userIds = positive_ratings['user'].unique()

ratings_train = ratings_train[ratings_train['user'].isin(positive_userIds)]
ratings_test = ratings_test[ratings_test['user'].isin(positive_userIds)]


### keep in test only ratings for movies which appear in train
ratings_test = ratings_test[ratings_test['item'].isin(ratings_train['item'])]

print(len(ratings_train['user'].unique()), 'users,', len(ratings_train['item'].unique()), 'items,', len(ratings_train.index), 'ratings in train set')

665 users, 93 items, 4390 ratings in train set


## Import the recommenders --- TO EDIT EXTERNALLY

You will implement the three recommenders from the previous assignments as classes in separate files. 

Steps:

1. Rename the three python files to `Recommender_CB.py`, `Recommender_CF_UU.py`, and `Recommender_MF.py`. That is, remove the ending `-TO_EDIT` from their names.

2. Fill in the missing code in these files. For the biggest part, you have to copy over the code you wrote for the assignment. IMPORTANT: because the recommenders are now implemented as separate classes, you may have to prefix non local variables with `self.` so that they are visible.

In general, the idea is to implement a simple recommender API:
```
class Recommender:  
    def __init__(self):
        pass
    
    def build_model(self, ratings_train, movies):
        pass
    
    def recommend(self, user_id, item_ids=None, topN=20):
        pass
```

Once done, we can import these classes.

In [19]:
from Recommender_CB import ContentBasedRecommender
from Recommender_CF_UU import UUCFRecommender
from Recommender_MF import MFRecommender

## Testing the recommenders

Make sure you have correctly copied the code from previous assignments to the right place.

In [86]:
cbr = ContentBasedRecommender('plot')
cbr.build_model(ratings_train, movies)

print(cbr.recommend(200))
print(cbr.recommend(100))
print(cbr.recommend(26))

[92 72 55 62 50 46 73 66 75 67 52 51 18 14 90 54 83 93 23 28]
[ 9  3 45 21 76 47 23 35 32 81  6 48 52 78 46 96 44  8 94 43]
[92 65 72 90 13 43 37 85 50 49 57 23 61  9 46 52 51 16 47 84]


**EXPECTED OUTPUT:**
```
[72, 55, 92, 50, 73, 66, 52, 46, 51, 18, 14, 67, 75, 93, 28, 45, 21, 83, 58, 23]
[9, 3, 45, 21, 76, 47, 35, 32, 81, 6, 23, 78, 48, 96, 8, 94, 52, 69, 43, 46]
[92, 65, 13, 43, 72, 37, 49, 57, 61, 46, 52, 50, 51, 16, 23, 84, 9, 83, 73, 89]
```

In [111]:
uucf = UUCFRecommender()
uucf.build_model(ratings_train)

print(uucf.recommend(200))
print(uucf.recommend(100))
print(uucf.recommend(26))

[97, 82, 90, 16, 37, 99, 94, 47, 14, 85, 50, 25, 58, 78, 73, 36, 62, 46, 7, 26]
[84, 40, 99, 30, 26, 73, 50, 37, 38, 17, 54, 47, 18, 63, 1, 41, 94, 81, 62, 6]
[37, 35, 90, 26, 49, 6, 85, 13, 58, 50, 99, 94, 25, 34, 36, 69, 14, 22, 40, 27]


/home/christoph/Dokumente/recsys_2019/assignment4/Recommender_CF_UU.py:166: RuntimeWarning: invalid value encountered in double_scalars
  neighborhood_weighted_avg = sumRatings / sumAbsWeights


**EXPECTED OUTPUT:**
```
[97, 82, 90, 16, 37, 99, 94, 47, 14, 85, 50, 25, 58, 78, 73, 36, 62, 46, 7, 26]
[84, 40, 99, 30, 26, 73, 50, 37, 38, 17, 54, 47, 18, 63, 1, 41, 94, 81, 62, 6]
[37, 35, 90, 26, 49, 6, 85, 13, 58, 50, 99, 94, 25, 34, 36, 69, 14, 22, 40, 27]
```

In [53]:
mfr = MFRecommender()
mfr.build_model(ratings_train)

print(mfr.recommend(200))
print(mfr.recommend(100))
print(mfr.recommend(26))

[50, 47, 6, 32, 36, 25, 58, 16, 62, 73, 72, 29, 21, 82, 92, 41, 84, 42, 56, 54]
[50, 47, 1, 17, 6, 32, 62, 73, 72, 11, 41, 29, 21, 92, 74, 13, 82, 76, 55, 7]
[50, 47, 6, 36, 58, 16, 25, 62, 82, 29, 72, 73, 42, 11, 56, 92, 41, 54, 96, 35]


**EXPECTED OUTPUT:**
```
[50, 47, 6, 32, 36, 25, 58, 16, 62, 73, 72, 29, 21, 82, 92, 41, 84, 42, 56, 54]
[50, 47, 1, 17, 6, 32, 62, 73, 72, 11, 41, 29, 21, 92, 74, 13, 82, 76, 55, 7]
[50, 47, 6, 36, 58, 16, 25, 62, 82, 29, 72, 73, 42, 11, 56, 92, 41, 54, 96, 35]
```

## Implement the Evaluator --- TO EDIT

The following class evaluates the ranking produced by a recommender.

**NOTE:** You should implement the following variant of DCG (there is a slight difference from the course slides):

$$\text{DCG}@k = \sum_{i=1}^k \frac{2^{rel_i}-1}{\log(i+1)}$$

where $rel_i$ is the relevance score of the item at position $i$ in the ranking. The definition of IDCG, and thus NDCG, is the same.

In [162]:
import math
import random
import warnings


DEBUG = True

if DEBUG:
    random.seed(42)

class Evaluator:
    
    def __init__(self, topN=20):
        self.topN = topN

    
    def init_data(self, ratings_train, ratings_test):
        self.ratings_train = ratings_train
        self.ratings_test = ratings_test
        self.find_unrated_items()
    
    
    ### store for each user her/his unrated items
    def find_unrated_items(self):
        all_items = set(self.ratings_train['item'].tolist())
        
        self.unrated = {}
        
        for user_id in self.ratings_train['user'].unique():
            rated_train_items = self.ratings_train[self.ratings_train['user'] == user_id]['item'].tolist()
            rated_test_items = self.ratings_test[self.ratings_test['user'] == user_id]['item'].tolist()

            rated_items = set(rated_train_items) | set(rated_test_items) # union of sets
            unrated_items = list(all_items - rated_items)
            random.shuffle(unrated_items)
            
            self.unrated[user_id] = unrated_items
            
    
    ### get the ratings of user_id in ratings_test
    def get_ground_truth(self, user_id):
        
        ## get the test ratings of user_id as a DataFrame subset of `self.ratings_test`
        # YOUR CODE HERE
        user_ratings = self.ratings_test.loc[self.ratings_test['user'] == user_id]
        
        ## dictionary of ground truth ratings
        ground_truth = pd.Series(user_ratings['rating'].values, index=user_ratings['item']).to_dict()

        return ground_truth
    
    
    def get_recommendations(self, model, user_id):
        ground_truth = self.get_ground_truth(user_id)
        n_test = len(ground_truth)
        
        ## we will create a total of topN items, and ask the recommender to rank them
        ## among these items, we will include the ground truth items
        
        ## 1. select (topN - n_test) unrated items
        item_ids = self.unrated[user_id][:self.topN - n_test]
        
        ## 2; add ground truth items
        item_ids = item_ids + list(ground_truth.keys())
        
        ## get the model's ranking
        recommendations = model.recommend(user_id, item_ids, self.topN)
        return recommendations
    
    
    ### evaluate the model on given user_id
    def eval_model_on_user(self, model, user_id):
        ground_truth = self.get_ground_truth(user_id)
        n_test = len(ground_truth)
        
        ## we will create a total of topN items, and ask the recommender to rank them
        ## among these items, we will include the ground truth items
        
        ## 1. select (topN - n_test) unrated items
        item_ids = self.unrated[user_id][:self.topN - n_test]
        
        ## 2; add ground truth items
        item_ids = item_ids + list(ground_truth.keys())
        
        ## get the model's ranking
        recommendations = model.recommend(user_id, item_ids, self.topN)
        
        ## evaluate the ranking
        metrics = self.get_ranking_metrics(ground_truth, recommendations)
        
        return metrics
    
    
    ### evaluate the model on all users
    def eval_model(self, model, n_users=-1):
        metrics_all = []
        count = 0;
        for user_id in self.ratings_train['user'].unique():
            count+=1
            print("\r", "evaluated on ", count, " users", end="", sep="")
            metrics = self.eval_model_on_user(model, user_id)
            if metrics is None:
                continue
            metrics_all.append(metrics)
            if count == n_users:
                break
        
        print("\n")
        
        
        ## store all metrics in a DataFrame for easy manipulation
        metrics_all_df = pd.DataFrame(metrics_all)
        self.metrics_all_df = metrics_all_df        
        
        ## average over all metrics
        hits_array = metrics_all_df.hits
        hits = np.nanmean(hits_array)
        ap_array = metrics_all_df.ap
        ap = np.nanmean(ap_array)
        
        rec_array = np.vstack(metrics_all_df.rec)
        prec_array = np.vstack(metrics_all_df.prec)
        ndcg_array = np.vstack(metrics_all_df.ndcg)
        
        
        with warnings.catch_warnings(): ## ignore division by 0
            warnings.simplefilter("ignore", category=RuntimeWarning)
            rec = np.nanmean(rec_array, axis=0)
            prec = np.nanmean(prec_array, axis=0)
            ndcg = np.nanmean(ndcg_array, axis=0)
        
        
        metrics_avg = {'hits':hits,
                   'ap':ap,
                   'rec':np.array(rec),
                   'prec':np.array(prec),
                   'ndcg':np.array(ndcg)}
        
        return metrics_avg
        
    ### get some evaluation metrics for ranking with respect to ground_truth
    def get_ranking_metrics(self, ground_truth, ranking):
        n_test = len(ground_truth)
        if n_test == 0:
            return None
        
        hits = 0 ## number of relevant in ranking
        rec = [] ## recall at every position of ranking
        prec = [] ## precision at every position of ranking
        dcg = [] ## DCG at every position of ranking
        ap = 0 ## average precision
        
        
        ## scan the ranking and compute hits, rec, prec, dcg, ap
        # YOUR CODE HERE
        runs = 1
        for x in ranking:
            if(x in ground_truth):
                hits+=1
                ap += (hits/runs)
                relevance = ground_truth.get(x)
            else:
                relevance = 0.
            prec.append(hits/runs)
            runs+=1
            numerator = math.pow(2,relevance) - 1
            denumerator = math.log(runs,2)
            if(len(dcg) > 0):
                dcg.append(np.array(dcg)[len(dcg)-1]+numerator/denumerator)
            else:
                dcg.append(numerator/denumerator)
                
        hitsForRecall = 0
        for x in ranking:
            if(x in ground_truth):
                hitsForRecall += 1
            rec.append(hitsForRecall/hits)
                
        if (hits != 0):
            ap /= hits
        else:
            ap = 0
            
        ## constuct the ideal ranking from ground truth to compute idcg
        ideal = sorted(ground_truth, key=ground_truth.get, reverse=True)
        idcg = []
        
        ## scan the ideal ranking and compute idcg
        # YOUR CODE HERE
        runs = 1
        for x in ideal:
            relevance = ground_truth.get(x)
            numerator = math.pow(2,relevance) - 1
            denumerator = math.log(runs+1,2)
            runs+=1
            if(len(idcg) > 0):
                idcg.append(np.array(idcg)[len(idcg)-1]+numerator/denumerator)
            else:
                idcg.append(numerator/denumerator)
                
        for x in range(0,(len(dcg)-len(idcg))):
            idcg.append(np.array(idcg)[len(idcg)-1])
            
        ## make sure the dcg and idcg lists have the same length
        if len(ideal) >= len(ranking):
            idcg = idcg[:len(ranking)]
        else:
            last_idcg = idcg[-1]
            for i in range(len(ranking) - len(ideal)):
                idcg.append(last_idcg)
        
        ## compute NDCG = DCG/IDCG
        ## TIP convert lists to `np.array` to do the division and then back to a list with `.tolist()`
        # YOUR CODE HERE
        
        ndcg = []
        for x in range(0,len(dcg)):
            ndcg.append(dcg[x]/idcg[x])
            
        rec = np.array(rec)
        prec = np.array(prec)
        ndcg = np.array(ndcg)
        
        ## make them have length self.topN, fill in with nan 
        rec = np.append(rec, np.repeat(np.nan, self.topN - len(rec)))
        prec = np.append(prec, np.repeat(np.nan, self.topN - len(prec)))
        ndcg = np.append(ndcg, np.repeat(np.nan, self.topN - len(ndcg)))
        
        metrics = {'hits':hits,
                   'ap':ap,
                   'rec':np.array(rec),
                   'prec':np.array(prec),
                   'ndcg':np.array(ndcg)}
        
        return metrics

## Test the ranking metrics

In [163]:
evl = Evaluator(topN = 10)

ground_truth = {200:5, 100:4, 400:3, 1000:3}

ranking = list(range(100, 1100, 100))

evl.get_ranking_metrics(ground_truth, ranking)

{'hits': 4,
 'ap': 0.7875,
 'rec': array([0.25, 0.5 , 0.5 , 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 1.  ]),
 'prec': array([1.        , 1.        , 0.66666667, 0.75      , 0.6       ,
        0.5       , 0.42857143, 0.375     , 0.33333333, 0.4       ]),
 'ndcg': array([0.48387097, 0.85406456, 0.78607189, 0.79980018, 0.79980018,
        0.79980018, 0.79980018, 0.79980018, 0.79980018, 0.84287192])}

**EXPECTED OUTPUT:**
```
{'ap': 0.7875,
 'hits': 4,
 'ndcg': array([0.48387097, 0.85406456, 0.78607189, 0.79980018, 0.79980018,
        0.79980018, 0.79980018, 0.79980018, 0.79980018, 0.84287192]),
 'prec': array([1.        , 1.        , 0.66666667, 0.75      , 0.6       ,
        0.5       , 0.42857143, 0.375     , 0.33333333, 0.4       ]),
 'rec': array([0.25, 0.5 , 0.5 , 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 1.  ])}
```

## Test the Evaluator

In [164]:
evl = Evaluator(topN = 20)

evl.init_data(ratings_train, ratings_test)

Show the ground truth for user 200

In [165]:
evl.get_ground_truth(200)

{48: 4.0, 62: 5.0}

Evaluate UU-CF on user 200:

In [166]:
evl.eval_model_on_user(uucf, 200)

{'hits': 2,
 'ap': 0.25,
 'rec': array([0. , 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. ]),
 'prec': array([0.        , 0.        , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.16666667, 0.15384615, 0.14285714, 0.13333333,
        0.125     , 0.11764706, 0.11111111, 0.10526316, 0.1       ]),
 'ndcg': array([0.        , 0.        , 0.38305705, 0.38305705, 0.38305705,
        0.38305705, 0.38305705, 0.38305705, 0.38305705, 0.38305705,
        0.38305705, 0.48323444, 0.48323444, 0.48323444, 0.48323444,
        0.48323444, 0.48323444, 0.48323444, 0.48323444, 0.48323444])}

**EXPECTED OUTPUT:**
```
{'ap': 0.25,
 'hits': 2,
 'ndcg': array([0.        , 0.        , 0.38305705, 0.38305705, 0.38305705,
        0.38305705, 0.38305705, 0.38305705, 0.38305705, 0.38305705,
        0.38305705, 0.48323444, 0.48323444, 0.48323444, 0.48323444,
        0.48323444, 0.48323444, 0.48323444, 0.48323444, 0.48323444]),
 'prec': array([0.        , 0.        , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.16666667, 0.15384615, 0.14285714, 0.13333333,
        0.125     , 0.11764706, 0.11111111, 0.10526316, 0.1       ]),
 'rec': array([0. , 0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. ])}
```

Evaluate MF on user 200:

In [167]:
evl.eval_model_on_user(mfr, 200)

{'hits': 2,
 'ap': 0.3026315789473684,
 'rec': array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. ]),
 'prec': array([0.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667,
        0.0625    , 0.05882353, 0.05555556, 0.10526316, 0.1       ]),
 'ndcg': array([0.        , 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.56913617, 0.56913617])}

**EXPECTED OUTPUT:**
```
{'ap': 0.3026315789473684,
 'hits': 2,
 'ndcg': array([0.        , 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.56913617, 0.56913617]),
 'prec': array([0.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333333, 0.07692308, 0.07142857, 0.06666667,
        0.0625    , 0.05882353, 0.05555556, 0.10526316, 0.1       ]),
 'rec': array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. ])}
```

Evaluate CB on user 200:

In [168]:
evl.eval_model_on_user(cbr, 200)

{'hits': 2,
 'ap': 0.3611111111111111,
 'rec': array([0. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 1. , 1. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. ]),
 'prec': array([0.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
        0.16666667, 0.14285714, 0.125     , 0.22222222, 0.2       ,
        0.18181818, 0.16666667, 0.15384615, 0.14285714, 0.13333333,
        0.125     , 0.11764706, 0.11111111, 0.10526316, 0.1       ]),
 'ndcg': array([0.        , 0.48336418, 0.48336418, 0.48336418, 0.48336418,
        0.48336418, 0.48336418, 0.48336418, 0.59495612, 0.59495612,
        0.59495612, 0.59495612, 0.59495612, 0.59495612, 0.59495612,
        0.59495612, 0.59495612, 0.59495612, 0.59495612, 0.59495612])}

**EXPECTED OUTPUT:**
```
{'ap': 0.18253968253968253,
 'hits': 2,
 'ndcg': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.12356679, 0.12356679, 0.35419012, 0.35419012,
        0.35419012, 0.35419012, 0.35419012, 0.35419012, 0.35419012,
        0.35419012, 0.35419012, 0.35419012, 0.35419012, 0.35419012]),
 'prec': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.14285714, 0.125     , 0.22222222, 0.2       ,
        0.18181818, 0.16666667, 0.15384615, 0.14285714, 0.13333333,
        0.125     , 0.11764706, 0.11111111, 0.10526316, 0.1       ]),
 'rec': array([0. , 0. , 0. , 0. , 0. , 0. , 0.5, 0.5, 1. , 1. , 1. , 1. , 1. ,
        1. , 1. , 1. , 1. , 1. , 1. , 1. ])}
```

Evaluate UU-CF on all users:

In [169]:
%time metrics = evl.eval_model(uucf)
display(metrics)

evaluated on 7 users

/home/christoph/.local/lib/python3.6/site-packages/scipy/sparse/base.py:597: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)
/home/christoph/Dokumente/recsys_2019/assignment4/Recommender_CF_UU.py:166: RuntimeWarning: invalid value encountered in double_scalars
  neighborhood_weighted_avg = sumRatings / sumAbsWeights


evaluated on 665 users

CPU times: user 17.5 s, sys: 115 ms, total: 17.6 s
Wall time: 17.4 s


{'hits': 1.9125874125874125,
 'ap': 0.220966208386088,
 'rec': array([0.02609058, 0.08760059, 0.14719794, 0.21321595, 0.27554806,
        0.33945915, 0.39639388, 0.48006299, 0.53507742, 0.59723402,
        0.64529915, 0.68993784, 0.74107004, 0.78454809, 0.83521895,
        0.87917707, 0.90714494, 0.9330149 , 0.95763403, 1.        ]),
 'prec': array([0.0541958 , 0.08653846, 0.0955711 , 0.10402098, 0.10874126,
        0.10984848, 0.10989011, 0.11363636, 0.11383061, 0.11363636,
        0.11252384, 0.11072261, 0.11013986, 0.10914086, 0.10862471,
        0.10675262, 0.1040724 , 0.10110723, 0.09817814, 0.09562937]),
 'ndcg': array([0.04760783, 0.08473862, 0.11742275, 0.14841456, 0.17578783,
        0.20137427, 0.22286995, 0.25240609, 0.27133172, 0.29028653,
        0.30576797, 0.31980926, 0.33346328, 0.3458532 , 0.358643  ,
        0.36917462, 0.37578013, 0.38224032, 0.38773944, 0.39719097])}

**EXPECTED OUTPUT:**
```
evaluated on 665 users

CPU times: user 5min 47s, sys: 4.91 s, total: 5min 52s
Wall time: 5min 53s
{'ap': 0.2209999986589639,
 'hits': 1.9125874125874125,
 'ndcg': array([0.04760783, 0.08473862, 0.11742275, 0.1491675 , 0.17654076,
        0.20150446, 0.22416565, 0.25204725, 0.2704466 , 0.29063027,
        0.3059012 , 0.31994249, 0.33267816, 0.34596304, 0.35586248,
        0.36751176, 0.37621354, 0.38216103, 0.38775957, 0.3972111 ]),
 'prec': array([0.0541958 , 0.08653846, 0.0955711 , 0.10445804, 0.10909091,
        0.10984848, 0.11038961, 0.11341783, 0.11344211, 0.11346154,
        0.11252384, 0.11072261, 0.1098709 , 0.10914086, 0.10780886,
        0.10631556, 0.10417524, 0.1010101 , 0.09817814, 0.09562937]),
 'rec': array([0.02609058, 0.08760059, 0.14719794, 0.2149642 , 0.27729631,
        0.33945915, 0.39989039, 0.47831474, 0.53158092, 0.59723402,
        0.64529915, 0.68993784, 0.73757354, 0.78454809, 0.82385531,
        0.87218407, 0.90889319, 0.93214078, 0.95763403, 1.        ])}
```

Evaluate MF on all users:

In [170]:
%time metrics = evl.eval_model(mfr)
display(metrics)

evaluated on 665 users

CPU times: user 2.74 s, sys: 78.3 ms, total: 2.82 s
Wall time: 2.7 s


{'hits': 1.9125874125874125,
 'ap': 0.45428173065156513,
 'rec': array([0.26044025, 0.39807415, 0.4548285 , 0.50302475, 0.53587315,
        0.55069999, 0.5699599 , 0.5879107 , 0.60330364, 0.62234571,
        0.64690448, 0.660724  , 0.67692446, 0.70301365, 0.72929987,
        0.75030039, 0.77767094, 0.8234564 , 0.89616148, 1.        ]),
 'prec': array([0.40734266, 0.3243007 , 0.25582751, 0.21678322, 0.18671329,
        0.16171329, 0.14310689, 0.12980769, 0.11926962, 0.11136364,
        0.10600763, 0.10052448, 0.09588488, 0.09303197, 0.09090909,
        0.08817745, 0.08669272, 0.08760684, 0.09054104, 0.09562937]),
 'ndcg': array([0.36942048, 0.42361113, 0.44563815, 0.46717659, 0.4793003 ,
        0.48431437, 0.4911899 , 0.49708216, 0.50225386, 0.50857961,
        0.51537594, 0.5195262 , 0.52435541, 0.53151516, 0.53846958,
        0.54346454, 0.55071635, 0.56230393, 0.57917726, 0.60118627])}

**EXPECTED OUTPUT:**
```
evaluated on 665 users

CPU times: user 2.83 s, sys: 365 ms, total: 3.2 s
Wall time: 2.93 s
{'ap': 0.45428173065156513,
 'hits': 1.9125874125874125,
 'ndcg': array([0.36942048, 0.42361113, 0.44563815, 0.46717659, 0.4793003 ,
        0.48431437, 0.4911899 , 0.49708216, 0.50225386, 0.50857961,
        0.51537594, 0.5195262 , 0.52435541, 0.53151516, 0.53846958,
        0.54346454, 0.55071635, 0.56230393, 0.57917726, 0.60118627]),
 'prec': array([0.40734266, 0.3243007 , 0.25582751, 0.21678322, 0.18671329,
        0.16171329, 0.14310689, 0.12980769, 0.11926962, 0.11136364,
        0.10600763, 0.10052448, 0.09588488, 0.09303197, 0.09090909,
        0.08817745, 0.08669272, 0.08760684, 0.09054104, 0.09562937]),
 'rec': array([0.26044025, 0.39807415, 0.4548285 , 0.50302475, 0.53587315,
        0.55069999, 0.5699599 , 0.5879107 , 0.60330364, 0.62234571,
        0.64690448, 0.660724  , 0.67692446, 0.70301365, 0.72929987,
        0.75030039, 0.77767094, 0.8234564 , 0.89616148, 1.        ])}
```

Evaluate CB on all users:

In [ ]:
%time metrics = evl.eval_model(cbr)
display(metrics)

**EXPECTED OUTPUT:**
```
evaluated on 665 users

CPU times: user 1.66 s, sys: 193 ms, total: 1.85 s
Wall time: 1.66 s
{'ap': 0.2574150703090204,
 'hits': 1.9125874125874125,
 'ndcg': array([0.1284034 , 0.14968061, 0.17719387, 0.20444699, 0.23179806,
        0.25068139, 0.26956727, 0.27856606, 0.29245607, 0.30326465,
        0.31470433, 0.32510047, 0.34102282, 0.35278245, 0.36588088,
        0.37989044, 0.39041599, 0.40218661, 0.41334282, 0.42952812]),
 'prec': array([0.15734266, 0.13636364, 0.12762238, 0.12325175, 0.12377622,
        0.11917249, 0.11638362, 0.10926573, 0.10528361, 0.10227273,
        0.09980928, 0.09717366, 0.09763314, 0.09615385, 0.095338  ,
        0.09538899, 0.09419992, 0.09411422, 0.09422157, 0.09562937]),
 'rec': array([0.07914655, 0.14004676, 0.19638417, 0.25784216, 0.32147644,
        0.37055098, 0.42190657, 0.44638209, 0.48709762, 0.52148893,
        0.56264361, 0.60004232, 0.65638875, 0.70191822, 0.74882548,
        0.80119811, 0.8411415 , 0.88661061, 0.93162879, 1.        ])}
```

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
#Tests

In [ ]:
#Tests